# Car Price Prediction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import metrics

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
car_dataset = pd.read_csv('./archive/train-data.csv')

In [ ]:
car_dataset.head()

In [ ]:
car_dataset.shape

In [ ]:
car_dataset.info()

In [ ]:
car_dataset.isnull().sum()

In [ ]:
print("Shape of train data Before dropping any Row: ",car_dataset.shape)
car_dataset = car_dataset[car_dataset['Mileage'].notna()]
print("Shape of train data After dropping Rows with NULL values in Mileage: ",car_dataset.shape)
car_dataset = car_dataset[car_dataset['Engine'].notna()]
print("Shape of train data After dropping Rows with NULL values in Engine : ",car_dataset.shape)
car_dataset = car_dataset[car_dataset['Power'].notna()]
print("Shape of train data After dropping Rows with NULL values in Power  : ",car_dataset.shape)
car_dataset = car_dataset[car_dataset['Seats'].notna()]
print("Shape of train data After dropping Rows with NULL values in Seats  : ",car_dataset.shape)

In [ ]:
car_dataset = car_dataset.reset_index(drop=True)

In [ ]:
for i in range(car_dataset.shape[0]):
    car_dataset.at[i, 'Company'] = car_dataset['Name'][i].split()[0]
    car_dataset.at[i, 'Mileage(km/kg)'] = car_dataset['Mileage'][i].split()[0]
    car_dataset.at[i, 'Engine(CC)'] = car_dataset['Engine'][i].split()[0]
    car_dataset.at[i, 'Power(bhp)'] = car_dataset['Power'][i].split()[0]

In [ ]:
car_dataset['Mileage(km/kg)'] = car_dataset['Mileage(km/kg)'].astype(float)
car_dataset['Engine(CC)'] = car_dataset['Engine(CC)'].astype(float)

In [ ]:
x = 'n'
count = 0
position = []
for i in range(car_dataset.shape[0]):
    if car_dataset['Power(bhp)'][i]=='null':
        x = 'Y'
        count = count + 1
        position.append(i)

In [ ]:
car_dataset = car_dataset.drop(car_dataset.index[position])
car_dataset = car_dataset.reset_index(drop=True)

In [ ]:
car_dataset.shape

In [ ]:
car_dataset['Power(bhp)'] = car_dataset['Power(bhp)'].astype(float)

In [ ]:
car_dataset.head()

In [ ]:
for i in range(car_dataset.shape[0]):
    if pd.isnull(car_dataset.loc[i,'New_Price']) == False:
        car_dataset.at[i,'New_car_Price'] = car_dataset['New_Price'][i].split()[0]

In [ ]:
car_dataset['New_car_Price'] = car_dataset['New_car_Price'].astype(float)

In [ ]:
car_dataset.head()

In [ ]:
car_dataset.drop(["Name"],axis=1,inplace=True)
car_dataset.drop(["Mileage"],axis=1,inplace=True)
car_dataset.drop(["Engine"],axis=1,inplace=True)
car_dataset.drop(["Power"],axis=1,inplace=True)
car_dataset.drop(["New_Price"],axis=1,inplace=True)

In [ ]:
car_dataset.info()

In [ ]:
car_dataset.head()

4 categorical features to check:

Location
Fuel_Type
Transmission
Owner_Type

In [ ]:
var = 'Seats'
car_dataset[var].value_counts()

In [ ]:
var = 'Location'
car_dataset[var].value_counts()

In [ ]:
# Plot Location vs Price
sns.catplot(y='Price',x=var,data= car_dataset.sort_values('Price',ascending=False),kind="boxen",height=6, aspect=3)
plt.show

One-hot-encoding here

In [ ]:
Location = car_dataset[[var]]
Location = pd.get_dummies(Location,drop_first=True)
Location.head()

In [ ]:
var = 'Fuel_Type'
car_dataset[var].value_counts()

In [ ]:
# Plot Fuel_Type vs Price
sns.catplot(y='Price',x=var,data= car_dataset.sort_values('Price',ascending=False),kind="boxen",height=6, aspect=3)
plt.show

One-hot-encoding here

In [ ]:
Fuel_t = car_dataset[[var]]
Fuel_t = pd.get_dummies(Fuel_t,drop_first=True)
Fuel_t.head()

In [ ]:
var = 'Transmission'
car_dataset[var].value_counts()

In [ ]:
# Plot Transmission vs Price
sns.catplot(y='Price',x=var,data= car_dataset.sort_values('Price',ascending=False),kind="boxen",height=6, aspect=3)
plt.show

One-hot-encoding here

In [ ]:
Transmission = car_dataset[[var]]
Transmission = pd.get_dummies(Transmission,drop_first=True)
Transmission.head()

In [ ]:
var = 'Owner_Type'
car_dataset[var].value_counts()

As Owner_Type column has ordered data so Label Encoding here

In [ ]:
car_dataset.replace({"First":1,"Second":2,"Third": 3,"Fourth & Above":4},inplace=True)
car_dataset.head()

In [ ]:
car_dataset.drop(["Company"],axis=1,inplace=True)

In [ ]:
final_train= pd.concat([car_dataset,Location,Fuel_t,Transmission],axis=1)
final_train.head()

In [ ]:
final_train.drop(["Location","Fuel_Type","Transmission","New_car_Price"],axis=1,inplace=True)
final_train.head()

In [ ]:
final_train.shape

In [ ]:
sns.pairplot(car_dataset)

In [ ]:
plt.figure(figsize=(18,18))
sns.heatmap(car_dataset.corr(),annot=True,cmap='RdYlGn')

plt.show()

# Final Features Selection

In [ ]:
final_train.columns

In [ ]:
X = final_train.loc[:,['Year', 'Kilometers_Driven', 'Owner_Type','Seats','Mileage(km/kg)','Engine(CC)','Power(bhp)', 
       'Location_Bangalore', 'Location_Chennai', 'Location_Coimbatore',
       'Location_Delhi', 'Location_Hyderabad', 'Location_Jaipur',
       'Location_Kochi', 'Location_Kolkata', 'Location_Mumbai',
       'Location_Pune', 'Fuel_Type_Diesel', 'Fuel_Type_LPG',
       'Fuel_Type_Petrol', 'Transmission_Manual']]
y = final_train.loc[:,['Price']]
y.head()

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
plt.figure(figsize=(18,18))
sns.heatmap(final_train.corr(),annot=True,cmap=sns.diverging_palette(20, 220, n=200))

plt.show()

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
selection= ExtraTreesRegressor()
selection.fit(X,y)

In [ ]:
plt.figure(figsize = (12,8))
feat_importances = pd.Series(selection.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)
y_pred= linear_reg.predict(X_test)
print("Accuracy on Traing set: ",linear_reg.score(X_train,y_train))
print("Accuracy on Testing set: ",linear_reg.score(X_test,y_test))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor()
dt_reg.fit(X_train, y_train)
y_pred= dt_reg.predict(X_test)
print("Accuracy on Traing set: ",dt_reg.score(X_train,y_train))
print("Accuracy on Testing set: ",dt_reg.score(X_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)
y_pred= rf_reg.predict(X_test)
print("Accuracy on Traing set: ",rf_reg.score(X_train,y_train))
print("Accuracy on Testing set: ",rf_reg.score(X_test,y_test))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(RandomForestRegressor(n_estimators=120, min_samples_split=2, min_samples_leaf=1, max_depth=20), X,y,cv=cv)

In [ ]:
cross_val_score(DecisionTreeRegressor(splitter='best', min_samples_leaf= 2, max_depth=15, criterion='mae'), X,y,cv=cv)

In [ ]:
predictions=rf_reg.predict(X_test)
plt.scatter(y_test,predictions)

In [ ]:
import pickle
with open('RF_price_predicting_model.pkl', 'wb') as file:
  # dump information to that file
  pickle.dump(rf_reg, file)